# ENV VARIABLES

In [1]:
DEPLOYMENT_NAME = 0

DATASET_NAMES = [
    "negative_feelings",
    "law_and_policy",
    "positive_feelings",
    "philosophy_and_ethics"
]

PATH = f"./datasets/dot_product/{DATASET_NAMES[DEPLOYMENT_NAME]}_with_explanations.csv"
# PATH = f"./datasets/dot_product/{}.csv"
print(PATH)

./datasets/dot_product/negative_feelings_with_explanations.csv


# INIT

In [2]:
import pandas as pd
import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import ast
import csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv(PATH)
df.head()

,type,layer,class1,class2,Neuron_ID,Contribution,Explanations
0,mlp,24,devastated,embarrassed,282,12699340.0,hyperlinks and web addresses
1,att,22,devastated,embarrassed,14953,12099597.0,attends to specific significant tokens from s...
2,att,12,devastated,embarrassed,2424,10867400.0,attends from tokens denoting an action or con...
3,mlp,24,embarrassed,furious,9329,10860906.0,legal terms and phrases related to court case...
4,att,7,ashamed,embarrassed,10364,10214085.0,attends to instances of the token followed by...


In [10]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from collections import defaultdict
from openai import AzureOpenAI
import json

# Azure credentials
AZURE_OPENAI_ENDPOINT = "https://interp.cognitiveservices.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME = "o4-mini"
AZURE_OPENAI_API_VERSION = "2024-12-01-preview"
AZURE_OPENAI_API_KEY = "BD3IG7AY4XibX8JjlehI5aKSXNVwtvsBopVPMDuRaoMHEdYSgGe6JQQJ99BGACYeBjFXJ3w3AAAAACOGxocP"

deployment_names = [
    "o4-mini",
    "o4-mini-2",
    "o4-mini-3",
    "o4-mini-4",
]

# Set env var for LangChain to use
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY

from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
# Initialize the LLM
# llm = AzureChatOpenAI(
#     azure_endpoint=AZURE_OPENAI_ENDPOINT,
#     azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
#     openai_api_version=AZURE_OPENAI_API_VERSION,
#     temperature=1.0,
#     model=deployment_names[DEPLOYMENT_NAME]
# )
# llm = make_llm(deployment_names[DEPLOYMENT_NAME])  # uses "o4-mini-X"
client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)


# Backgrounds
# Backgrounds
DATASET_BACKGROUND = {
    "mmlu": """
SYSTEM:
  You are an evaluator agent analyzing skills for neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses academic or professional domain knowledge.
  These concepts include any of these specific topics: abstract algebra, anatomy, astronomy, business ethics, clinical knowledge, college biology, college chemistry, college computer science, college mathematics, college medicine, computer security, conceptual physics, econometrics, electrical engineering, elementary mathematics, formal logic, global facts, high school biology, high school chemistry, high school computer science, high school European history, high school geography, high school government and politics, high school macroeconomics, high school mathematics, high school microeconomics, high school physics, high school psychology, high school statistics, high school US history, high school world history, human aging, human sexuality, international law, jurisprudence, logical fallacies, machine learning, management, marketing, medical genetics, miscellaneous, moral disputes, moral scenarios, nutrition, philosophy, prehistory, professional accounting, professional law, professional medicine, professional psychology, public relations, security studies, sociology, US foreign policy, virology, world religions.
  It also includes any general references to domain-specific facts, academic reasoning, professional terminology, or subject-matter expertise.

INSTRUCTIONS:
1. Think step-by-step, identifying any cues of code structure, algorithmic reasoning, or programming terminology.
2. EXCLUDE any explanations that talk only about low-level token patterns, lexical tokens, or sequence-level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of domain knowledge.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step by step reasoning goes here.",
  "decision": "yes | no"
}
```""",

    "empathetic_dialogue": """
SYSTEM:
  You are an evaluator agent analyzing skills for neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses emotional understanding, emotional reasoning, positive or negative emotion.
  These concepts include any of these specific topics: afraid, angry, annoyed, anticipating, anxious, apprehensive, ashamed, caring, confident, content, devastated, disappointed, disgusted, embarrassed, excited, faithful, furious, grateful, guilty, hopeful, impressed, jealous, joyful, lonely, nostalgic, prepared, proud, sad, sentimental, surprised, terrified, trusting.
  It also includes any general references to emotional distress, negative emotional states, psychological discomfort, or interpersonal sensitivity.

INSTRUCTIONS:
1. Think step-by-step, identifying any cues of code structure, algorithmic reasoning, or programming terminology.
2. EXCLUDE any explanations that talk only about low-level token patterns, lexical tokens, or sequence-level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of domain knowledge.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step by step reasoning goes here.",
  "decision": "yes | no"
}
```""",

    "math": """
SYSTEM:
  You are an evaluator agent analyzing skills for neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses mathematical reasoning or problem-solving ability.
  These topics include any of these specific areas: algebra, prealgebra, intermediate algebra, number theory, precalculus, geometry, counting and probability.
  It also includes any general references to numerical reasoning, arithmetic operations, solving equations, geometric understanding, or probabilistic logic.

INSTRUCTIONS:
1. Think step-by-step, identifying any cues of code structure, algorithmic reasoning, or programming terminology.
2. EXCLUDE any explanations that talk only about low-level token patterns, lexical tokens, or sequence-level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of domain knowledge.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step by step reasoning goes here.",
  "decision": "yes | no"
}
```""",

    "programming": """
SYSTEM:
  You are an evaluator agent analyzing skills for neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses programming knowledge or computational reasoning.
  These concepts include any of these specific topics: programming syntax, programming languages (e.g., Python, C++, JavaScript), data structures (e.g., arrays, lists, trees, graphs), algorithms (e.g., sorting, searching, dynamic programming), code logic, function definitions, debugging steps, pseudocode, object-oriented programming, recursion, loops, conditions, and basic software engineering concepts.
  It also includes any general references to code structure, implementation logic, computational flow, or language-specific behavior.

INSTRUCTIONS:
1. Think step-by-step, identifying any cues of code structure, algorithmic reasoning, or programming terminology.
2. EXCLUDE any explanations that talk only about low-level token patterns, lexical tokens, or sequence-level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of domain knowledge.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step by step reasoning goes here.",
  "decision": "yes | no"
}
```""",
}


WITHIN_SKILL_DATASET = {
    "negative_feelings": """
SYSTEM:
        You are an evaluator agent analyzing neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses negative emotion or negative sentiment.
        
        Negative emotional concepts include any of these specific feelings:
        afraid, angry, annoyed, anxious, apprehensive, ashamed, devastated, disappointed, disgusted, embarrassed, furious, guilty, jealous, lonely, sad, terrified.
        
        It also includes any general references to negative sentiment, negative feelings, distress, negativity, or emotional conflict/responses.
        
        INSTRUCTIONS:
        1. Think step-by-step, identifying any cues of negative emotion or sentiment.
        2. If there is any uncertainty, choose “no”.
        3. Output only in JSON (no extra text, no explanation outside your rationale field)

    Provide your reasoning and final decision in this exact JSON format:    
    ```json
    {{
      "rationale": "Your step by step reasoning goes here.",
      "decision": "yes | no"
    }}
    ```
    """,

    "positive_feelings": """
    SYSTEM:
You are an evaluator agent analyzing neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto-Interpretability system. Your task is to judge whether the explanation expresses positive emotion or positive sentiment.

Positive emotional concepts include any of these specific feelings:
caring, confident, content, excited, faithful, grateful, hopeful, impressed, joyful, prepared, proud, surprised, trusting.

It also includes any general references to positive sentiment, positive feelings, positivity, uplift, optimism, or emotional well-being.

INSTRUCTIONS:
1. Think step-by-step, identifying any cues of positive emotion or positive sentiment.
2. EXCLUDE any explanations that talk only about low-level token patterns, lexical tokens, or sequence‑level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of positive emotion.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step-by-step reasoning goes here.",
  "decision": "yes | no"
}
""",
    "law_and_policy":"""
    SYSTEM:
You are an evaluator agent analyzing neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto‑Interpretability system. Your task is to judge whether the explanation expresses legal expertise or knowledge.

Legal‐expertise concepts include any of these specific topics:
jurisprudence, professional law, international law, U.S. foreign policy, security studies.

It also includes any general references to legal reasoning, statutory interpretation, case law analysis, treaties, regulatory frameworks, or national/international security policy.

INSTRUCTIONS:
1. Think step‑by‑step, identifying any cues of legal expertise or knowledge.
2. EXCLUDE any explanations that talk only about low‑level token patterns, lexical tokens, or sequence‑level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of legal understanding.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step‑by‑step reasoning goes here.",
  "decision": "yes | no"
}
""",
    "philosophy_and_ethics": """
    SYSTEM:
You are an evaluator agent analyzing neuron activations on Sparse Autoencoders. You will receive an explanation from an Auto‑Interpretability system. Your task is to judge whether the explanation expresses **philosophical or ethical reasoning**.

Philosophy‑and‑ethics concepts include any of these specific topics:
philosophy, moral disputes, moral scenarios, world religions.

It also includes any general references to ethical principles, normative theories, values-based reasoning, metaphysical inquiry, theological perspectives, or moral judgment.

INSTRUCTIONS:
1. Think step‑by‑step, identifying any cues of philosophical or ethical reasoning.
2. EXCLUDE any explanations that talk only about low‑level token patterns, lexical tokens, or sequence‑level formatting (e.g., “attends to verbs…”, “counts tokens…”, “looks at underscores…”). Those are not evidence of philosophical or ethical understanding.
3. If there is any uncertainty, choose “no”.
4. Output only in JSON (no extra text, no explanation outside your rationale field).

Provide your reasoning and final decision in this exact JSON format:
```json
{
  "rationale": "Your step‑by‑step reasoning goes here.",
  "decision": "yes | no"
}

""" 
}

In [11]:
import json

# def parse_llm_response(response):
#     # Extract string from response.content
#     raw_content = response.content.strip()

#     # Remove triple backticks and possible "json" label
#     if raw_content.startswith("```json"):
#         raw_content = raw_content[7:]  # remove ```json\n
#     if raw_content.endswith("```"):
#         raw_content = raw_content[:-3]  # remove trailing ```

#     # Clean whitespace and load as JSON
#     return json.loads(raw_content.strip())

# def parse_llm_response(response) -> dict:
#     # Azure returns a ChatCompletions model with .choices,
#     # each having .message.content
#     raw_content = response.choices[0].message.content.strip()

#     # strip triple-backticks if present
#     if raw_content.startswith("```"):
#         # drop ```json or ```
#         raw_content = raw_content.split("```", 1)[1].rsplit("```", 1)[0].strip()

#     # now load JSON
#     return json.loads(raw_content)

def parse_llm_response(response) -> dict:
    # 1) Extract the content field (guarding against None or missing)
    try:
        choice = response.choices[0]
        content = getattr(choice.message, "content", None)
        if not content:
            raise ValueError("No content returned by LLM")
        raw = content.strip()
    except (IndexError, AttributeError, ValueError) as e:
        # Something went wrong extracting the text
        raise RuntimeError(f"Failed to extract LLM content: {e}")

    # 2) Remove ``` fences if present
    if raw.startswith("```"):
        # split out the fenced block
        parts = raw.split("```")
        # parts[0] is before the first fence (likely empty), parts[1] is language (maybe "json") or content
        # parts[2] is the inner content if fenced
        raw = parts[2] if len(parts) >= 3 else parts[1]

    raw = raw.strip()

    # 3) JSON-parse, but catch decode errors
    try:
        return json.loads(raw)
    except json.JSONDecodeError as e:
        # include the raw string in the exception for debugging
        raise ValueError(
            f"Failed to parse JSON (char {e.pos}): {e.msg}\n"
            f"Raw content was:\n{raw}"
        )

In [12]:

def get_answer(QUESTION: str, print_output: bool = False):
    # System prompt with escaped braces for JSON formatting
    system_template = WITHIN_SKILL_DATASET[DATASET_NAMES[DEPLOYMENT_NAME]]
    # system_template = WITHIN_SKILL_DATASET[DATASET_NAMES[DEPLOYMENT_NAME]]

    # User message prompt
    user_prompt = PromptTemplate(
        input_variables=["question"],
        template="""
        Question:
        {question}
        """
    )

    user_message = user_prompt.format(question=QUESTION)

    # LLM call
    # response = llm.invoke([
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_template},
            {"role": "user", "content": user_message}
        ],
        model= deployment_names[DEPLOYMENT_NAME]                                        
    )
    # print(system_template)
    return parse_llm_response(response)
    # return parse_json_response(response.content, print_output=print_output)


In [13]:
# skill_name = "terrified"
# skill_examples = """
#         * Emotional expressions 
#         * Mentions words or concepts tied to emotional states of fear, anxiety, panic, afraid or stress.
#         * Describes reactions or content commonly associated with being terrified (e.g., phrases evoking danger, emotional distress, intense fear).
#         * Suggests personal or emotional responses aligned with fear or negativity.
#         * Terms that can result in a terrified state
# """
# ============================================================================

# question = "expressions of personal sentiment and emotional responses"
# question = "I am so scared of that happening"
# question = "elements related to conflict and action"
# question = "expressions of uncertainty and emotional reactions"
# question = "negations and expressions implying uncertainty or disagreement"
# question = "intensifiers related to feelings or emotions"
# question = "expressions of emotional states and interpersonal connections"
# question = "expressions of strong emotions or feelings"
# question = "expressions of love and enjoyment of outdoor activities"
# question = "emotions and introspective thoughts related to intense experiences"
# question = "references to family relationships and their emotional connections"
# question = "emotional expressions and states"
# question = "phrases that convey experiences of distress or psychological states"
# question = "references to negative thoughts"
# question = "negative emotional states or feelings associated with difficult situations"
# question = "expressions of fear and anxiety"
# question = "expressions and sentiments related to love and affection"
# question = "phrases indicating personal struggle or emotional conflict"
# question = "expressions of fear and anxiety"
question = "attends to descriptive or evaluative adjectives and phrases indicating qualities or states from subjective expressions or comments	"

output = get_answer(question)

# print(output)
print(output["rationale"])
print(output["decision"])

The given text is a description of a system component attending to descriptive or evaluative adjectives and phrases. It contains no expression of negative emotion or sentiment, nor any of the listed negative emotional concepts.
no


# DECISION

In [54]:
# import pandas as pd
# from tqdm import tqdm

# # make sure the column exists
# if "llm_decision" not in df.columns:
#     df["llm_decision"] = None

# for idx, row in tqdm(df.iterrows(), total=len(df), desc="LLM EVAL"):
#     explanation = row["Explanation"]
#     try:
#         response = get_answer(
#             QUESTION=explanation,
#             print_output=False
#         )
#         df.at[idx, "llm_decision"] = response["decision"]
#         # df.at[idx, "llm_rationale"] = response["rationale"]
#     except Exception as e:
#         df.at[idx, "llm_decision"] = "ERROR"

# df.to_csv(path, index=False)


In [55]:
COL_NAME = DATASET_NAMES[DEPLOYMENT_NAME]
COL_NAME

'negative_feelings'

In [56]:
df.shape[0]

220210

In [57]:
df.head()

,type,layer,class1,class2,Neuron_ID,Contribution,Explanations
0,mlp,24,devastated,embarrassed,282,12699340.0,hyperlinks and web addresses
1,att,22,devastated,embarrassed,14953,12099597.0,attends to specific significant tokens from s...
2,att,12,devastated,embarrassed,2424,10867400.0,attends from tokens denoting an action or con...
3,mlp,24,embarrassed,furious,9329,10860906.0,legal terms and phrases related to court case...
4,att,7,ashamed,embarrassed,10364,10214085.0,attends to instances of the token followed by...


In [58]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# --- setup (as before) ---
# df is already loaded
# COL_NAME, DATASET_NAMES, DEPLOYMENT_NAME, get_answer are defined

if COL_NAME not in df.columns:
    df[COL_NAME] = None

def process_row(idx, explanation, max_retries=2, backoff_s=1):
    for attempt in range(max_retries + 1):
        try:
            resp = get_answer(QUESTION=explanation, print_output=False)
            return idx, resp["decision"]
        except Exception:
            if attempt < max_retries:
                time.sleep(backoff_s)
            else:
                return idx, "ERROR"

# how far we want to go
new_max = df.shape[0]
# new_max = 5

# pick only those in [0, new_max) still unfilled
to_process = (
    df
    .iloc[:new_max]
    .loc[df[COL_NAME].isna()]
    .iterrows()
)
total = len(df.iloc[:new_max]) - df[COL_NAME].notna().sum()
print(f"Resuming: {total} rows to go (up to {new_max})")

output_name = DATASET_NAMES[DEPLOYMENT_NAME]
output_path = f"./datasets/dot_product/explanations_{output_name}.csv"

# counter for checkpointing
processed_since_start = 0

with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {
        executor.submit(process_row, idx, row["Explanations"]): idx
        for idx, row in to_process
    }

    for future in tqdm(as_completed(futures), total=total, desc="Resuming LLM EVAL"):
        idx, decision = future.result()
        # write back immediately
        df.at[idx, COL_NAME] = decision

        processed_since_start += 1
        # every 10k, flush to disk
        if processed_since_start % 10_000 == 0:
            print(f"[{processed_since_start}/{total}] rows done — saving checkpoint...")
            df.to_csv(output_path, index=False)

# final save
df.to_csv(output_path, index=False)
print("All done! Final CSV written to", output_path)

Resuming: 5 rows to go (up to 5)


Resuming LLM EVAL: 100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


All done! Final CSV written to ./datasets/explanations_negative_feelings.csv


In [38]:
df.head(70)

,type,layer,class1,class2,Neuron_ID,Contribution,Explanation,negative_feelings
0,mlp,24,devastated,embarrassed,282,12699340.0,hyperlinks and web addresses,no
1,att,22,devastated,embarrassed,14953,12099597.0,attends to specific significant tokens from s...,no
2,att,12,devastated,embarrassed,2424,10867400.0,attends from tokens denoting an action or con...,no
3,mlp,24,embarrassed,furious,9329,10860906.0,legal terms and phrases related to court case...,no
4,att,7,ashamed,embarrassed,10364,10214085.0,attends to instances of the token followed by...,no
5,res,1,ashamed,embarrassed,12054,9616475.0,content related to medical conditions and trea...,None
6,att,23,devastated,embarrassed,5279,9500750.0,"attends to the token ""by"" from the surroundin...",None
7,mlp,7,embarrassed,furious,6117,9141438.0,topics related to nutrition and health measur...,None
8,att,5,embarrassed,furious,9670,9120768.0,attends to specific tokens lacking any conten...,None
9,res,12,embarrassed,furious,2620,8510100.0,"words associated with authority, control, and ...",None
